<a href="https://colab.research.google.com/github/kevin1109500/MACHINE-LEARNING/blob/main/DIABETES_PREDICTION_DUE_TO_OBESITY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lazypredict

In [ ]:
import pandas as pd
import warnings
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import r2_score,accuracy_score,precision_score,recall_score,classification_report,confusion_matrix,f1_score

import lazypredict
from lazypredict.Supervised import LazyClassifier

%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# reading data from excel file named obesity project
df = pd.read_excel('/content/obesity project (4).xls',sheet_name='new trial group')
df.head()

In [ ]:
print("Size of Dataset: {} rows , {} columns".format(df.shape[0],df.shape[1]))

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

In [ ]:
def Missing_Values(data):
    variable_name = []
    total_value = []
    total_missing_value = []
    missing_value_rate = []
    unique_value_list = []
    total_unique_value = []
    data_type = []

    for col in data.columns:
        variable_name.append(col)
        data_type.append(data[col].dtype)
        total_value.append(data[col].shape[0])
        total_missing_value.append(data[col].isnull().sum())
        missing_value_rate.append(round(data[col].isnull().sum()/data[col].shape[0],4))
        unique_value_list.append(data[col].unique())
        total_unique_value.append(len(data[col].unique()))

    missing_data=pd.DataFrame({"Variable":variable_name,\
                               "#_Total_Value":total_value,\
                               "#_Total_Missing_Value":total_missing_value,\
                               "%_Missing_Value_Rate":missing_value_rate,\
                               "Data_Type":data_type,"Unique_Value":unique_value_list,\
                               "Total_Unique_Value":total_unique_value
                              })

    missing_data = missing_data.set_index("Variable")
    return missing_data.sort_values("#_Total_Missing_Value",ascending=False)

In [ ]:
Missing_Values(df)

In [ ]:
msno.matrix(df)
plt.show()

In [ ]:
msno.bar(df)
plt.show()

In [ ]:
# checking for correlation bet the features with null values
msno.heatmap(df)
plt.show()

In [ ]:
df = df.drop(['after intervention triglycerides', 'after intervention Total cholestrol', 'after intervention ldl',
              'after intervention', 'trial HBA1C', 'AFTER INTERVENTION  cho intake', 'AFTER INTERVENTION PROTEIN INTAKE',
              'after intervention calories', 'AFTER INTERVENTION  fat intake', 'after trial FBS', 'after trial PPBS',
              'after intervention BMI (Kg/m2)'], axis = 1)

df = df.drop(['Prescribed Calories','patient intake Calories','protein intake','cho intake','fat intake','follow up wt','remarks'], axis = 1)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
# replacing null values with median of respectively column median
df[['baseline FBS', 'PPBS','baseline HBA1C','Total cholestrol','triglycerides','hdl','ldl']] = df[['baseline FBS', 'PPBS','baseline HBA1C','Total cholestrol','triglycerides','hdl','ldl']].fillna(df[['baseline FBS', 'PPBS','baseline HBA1C','Total cholestrol','triglycerides','hdl','ldl']].median())

In [ ]:
print("Size of Dataset Before dropping the null values: {} rows , {} columns".format(df.shape[0],df.shape[1]+18))
df.dropna(axis=0,inplace = True)
print("Size of Dataset After dropping the null values: {} rows , {} columns".format(df.shape[0],df.shape[1]))

In [ ]:
df.columns

In [ ]:
df['Age (yrs)'].value_counts()

In [ ]:
df['Diagnosis'].value_counts()

In [ ]:
df['Diagnosis'] = df['Diagnosis'].replace(['diabetes mellitus,obese class1'], 'diabetes mellitus obese class 1')
df['Diagnosis'] = df['Diagnosis'].replace(['diabetic mellitus,obese class1'], 'diabetes mellitus obese class 1')
df['Diagnosis'] = df['Diagnosis'].replace(['diabetic mellitus,obese class2'], 'diabetes mellitus obese class 2')
df['Diagnosis'] = df['Diagnosis'].replace(['diabetic mellitus,obese grade 2'], 'diabetes mellitus obese class 2')

In [ ]:
df['Diagnosis'].value_counts()

In [ ]:
df['job type'].value_counts()

In [ ]:
df.drop(["job type"],axis=1,inplace = True)

In [ ]:
df['physical activity'].value_counts()

In [ ]:
df['physical activity'] = df['physical activity'].replace(['walking 1/2hr','walking 30min/day','walking1/2hr','walking 35mins a day','walking daily 1/2hr','walking 1/2hr,hypoglycemia','walking 30mins'], 'walking 30 mins')
df['physical activity'] = df['physical activity'].replace(['walking 1 hr/day','walking 1hr','swimming 1hr','walking 1hr/day','walking 50mins a day'], 'walking 1 hr')
df['physical activity'] = df['physical activity'].replace(['walking 10-15mins','walking 20mins/day','walking 20min/day'], 'walking 15 mins')
df['physical activity'] = df['physical activity'].replace(['walking 45mins','walking 45min/day','walking 40mins/day','walking 45-1hr','walking 45mins a day'], 'walking 45 mins')
df['physical activity'] = df['physical activity'].replace(['walking 1hr weekly 3 times','walking weekly \nthree times 20mins'], 'walking 3hr weekly')
df['physical activity'] = df['physical activity'].replace(['exercise for 1hr'], 'exercise 1hr')

In [ ]:
df['physical activity'].value_counts()

In [ ]:
df.drop(["physical activity"],axis=1,inplace = True)

In [ ]:
numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
for cols in numeric_columns:
  if cols != 'Sex':
    print(f"\n\n {cols} \n")
    mean = np.mean(df[cols])
    std = np.std(df[cols])
    print('mean is', mean)
    print('std. deviation is', std)
    threshold = 3
    outlier = []
    for i in df[cols]:
        z = (i-mean)/std
        if z > threshold:
            outlier.append(i)
    if len(outlier) != 0:
      print('outliers are', outlier)
    else:
      print("No Outliers")

In [ ]:
numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
for cols in numeric_columns:
  if cols != 'Sex':
    x = math.floor(df[cols].quantile(0.00))
    y = math.floor(df[cols].quantile(0.95))

    df[cols] = np.where(df[cols] < x, x,df[cols])
    df[cols] = np.where(df[cols] > y, y,df[cols])

In [ ]:
df1 = df.copy(deep=True)

In [ ]:
df1['Diagnosis'] = pd.Categorical(df1['Diagnosis']).codes
df1['Diagnosis'].value_counts()

In [ ]:
df_corr = df1.corr() # Generate correlation matrix
x = list(df_corr.columns)
y = list(df_corr.index)
z = np.array(df_corr)

fig = ff.create_annotated_heatmap(
    z,
    x = x,
    y = y,
    annotation_text = np.around(z, decimals=2),
    hoverinfo='z',
    colorscale='Viridis',
    zmax = 1,
    zmin = -1
    )
fig.show()

In [ ]:
df['Diagnosis'].value_counts()

In [ ]:
df2 = df.copy(deep=True)

In [ ]:
df2 = pd.get_dummies(df2, columns = ['Diagnosis'])

In [ ]:
df_corr = df2.corr() # Generate correlation matrix
x = list(df_corr.columns)
y = list(df_corr.index)
z = np.array(df_corr)

fig = ff.create_annotated_heatmap(
    z,
    x = x,
    y = y,
    annotation_text = np.around(z, decimals=2),
    hoverinfo='z',
    colorscale='Viridis',
    zmax = 1,
    zmin = -1
    )
fig.show()

In [ ]:
df['Diagnosis'] = pd.Categorical(df['Diagnosis']).codes

In [ ]:
df.head()

In [ ]:
y = df['Diagnosis']
x = df.drop(['Diagnosis'],axis = 1)

In [ ]:
# our data is a imbalanced data
y.value_counts().plot(kind = 'bar')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    stratify=y,
                                                    test_size=0.25)

In [ ]:
print(y_train.value_counts(),y_test.value_counts())

In [ ]:
for cols in X_train.columns:
  if cols != 'Sex':
    X_train[cols] = StandardScaler().fit_transform(np.array(X_train[cols]).reshape(-1, 1))
    X_test[cols] = StandardScaler().fit_transform(np.array(X_test[cols]).reshape(-1, 1))

In [ ]:
print("Before OverSampling, counts of label '2': {}".format(sum(y_train == 2)))
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))


sm = SMOTE(random_state = 42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '2': {}".format(sum(y_train_res == 2)))
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

In [ ]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)

models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

In [ ]:
xgbc = XGBClassifier()

xgbc.fit(X_train_res, y_train_res)
y_pred = xgbc.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
print(classification_report(y_pred,y_test))

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0]
}

# Initialize XGBClassifier
xgb = XGBClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='accuracy')

# Fit GridSearchCV to training data
grid_search.fit(X_train_res, y_train_res)

# Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

In [ ]:
xgbc = XGBClassifier(learning_rate =  0.01, max_depth = 5, min_child_weight = 1, n_estimators = 100, subsample = 0.8)

xgbc.fit(X_train_res, y_train_res)
y_pred = xgbc.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
print(classification_report(y_pred,y_test))

In [ ]:
classifier = RandomForestClassifier()

classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
print(classification_report(y_pred,y_test))

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced', {0: 0.3, 1: 0.7}]
}

# Initialize RandomForestClassifier
Classifier = RandomForestClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=Classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Fit GridSearchCV to training data
grid_search.fit(X_train_res, y_train_res)

# Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")


In [ ]:
classifier = RandomForestClassifier(class_weight = None, max_depth = 5, max_features = 'sqrt', min_samples_split = 2, n_estimators = 100)

classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
sns.heatmap(confusion_matrix(y_pred,y_test), annot = True)
plt.show()

In [ ]:
classifier = LogisticRegression()

classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
sns.heatmap(confusion_matrix(y_pred,y_test), annot = True)
plt.show()

In [ ]:
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [50, 100, 200],
    'class_weight': [None, 'balanced', {0: 0.3, 1: 0.7}]
}

# Initialize LogisticRegression
Classifier = LogisticRegression()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=Classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Fit GridSearchCV to training data
grid_search.fit(X_train_res, y_train_res)

# Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

In [ ]:
classifier = LogisticRegression(C = 1, class_weight = {0: 0.3, 1: 0.7}, max_iter = 100, penalty = 'l1', solver = 'saga')

classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
sns.heatmap(confusion_matrix(y_pred,y_test), annot = True)
plt.show()

In [ ]:
classifier =  KNeighborsClassifier()

classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
sns.heatmap(confusion_matrix(y_pred,y_test), annot = True)
plt.show()

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2, 3],
    'leaf_size': [20, 30, 40]
}

# Initialize KNeighborsClassifier
Classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=Classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Fit GridSearchCV to training data
grid_search.fit(X_train_res, y_train_res)

# Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

In [ ]:
classifier = SVC()

classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
sns.heatmap(confusion_matrix(y_pred,y_test), annot = True)
plt.show()

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4],
    'class_weight': [None, 'balanced'],
    'gamma': ['scale', 'auto', 0.1, 1]
}

# Initialize KNeighborsClassifier
Classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=Classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Fit GridSearchCV to training data
grid_search.fit(X_train_res, y_train_res)

# Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

In [ ]:
classifier = SVC(C = 1, class_weight = None, degree = 2, gamma = 'scale', kernel = 'linear')

classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
sns.heatmap(confusion_matrix(y_pred,y_test), annot = True)
plt.show()

In [ ]:
model = XGBClassifier(learning_rate =  0.01, max_depth = 5, min_child_weight = 1, n_estimators = 100, subsample = 0.8)

model.fit(X_train_res, y_train_res)
y_pred = model.predict(X_test)

print("Accuracy score : ",accuracy_score(y_pred,y_test))
print("r2 score : ",r2_score(y_pred,y_test))
print("precision score : ",precision_score(y_pred,y_test,average='micro'))
print("recall score : ",recall_score(y_pred,y_test,average='micro'))
print("f1 score : ",f1_score(y_pred,y_test,average='micro'))
print("\nclassification report \n")
sns.heatmap(confusion_matrix(y_pred,y_test), annot = True)
plt.show()